In [1]:
pip install networkx

In [2]:
import numpy as np
from numpy.linalg import matrix_power
import math as m
import networkx as nx
from numpy.linalg import matrix_power
import random
import pandas as pd


In [3]:
def RobustGC(W, y_train_prime, eta):
  D = np.zeros ((len(W),len(W)))
  for i in range (len(W)):
    D[i][i]= np.sum(W[i] , axis=0)

  D_inv = np.sqrt(matrix_power(D,-1))
  S = D_inv @ W @ D_inv
  LN = np.eye(len(S)) - S
  V0 = np.zeros(len(D))

  for i in range (len (D)):
    V0[i]= np.sqrt(D[i][i])

  V0_norm = np.linalg.norm(V0, 2)
  V0= V0/V0_norm
  v=np.linalg.eigvals(LN)
  v=sorted(v)
  lambda_1 = v[1]
  gamma_t= eta*lambda_1
  # print(gamma_t)
  
  f = np.linalg.inv((LN/gamma_t)-np.eye(len(LN)))@(y_train_prime - (np.dot(V0,np.dot(V0.transpose(), y_train_prime))))
  pred= np.sign(f)
  pred = np.squeeze(np.asarray(pred))
  # print (pred)

  return pred, gamma_t

In [4]:
def PF_RobustGC(W, y_train_prime):
  D = np.zeros ((len(W),len(W)))
  for i in range (len(W)):
    D[i][i]= np.sum(W[i] , axis=0)

  D_inv = np.sqrt(matrix_power(D,-1))
  S = D_inv @ W @ D_inv
  LN = np.eye(len(S)) - S
  V0 = np.zeros(len(D))

  for i in range (len (D)):
    V0[i]= np.sqrt(D[i][i])

  V0_norm = np.linalg.norm(V0, 2)
  V0= V0/V0_norm
  v=np.linalg.eigvals(LN)
  v=sorted(v)
  lambda_1 = v[1]
  gamma_t= 0.9*lambda_1
  # print(gamma_t)
  
  f = np.linalg.inv((LN/gamma_t)-np.eye(len(LN)))@(y_train_prime - (np.dot(V0,np.dot(V0.transpose(), y_train_prime))))
  pred= np.sign(f)
  pred = np.squeeze(np.asarray(pred))
  # print (pred)

  return pred, gamma_t

In [5]:
def accuracy (y_pred , y_orig):
  count=0
  for i in range (len(y_pred)):
    if y_pred[i]==y_orig[i]:
      count+=1
  result = count/len(y_pred) * 100
  return result

In [6]:
data = nx.read_gml('/content/polbooks.gml', label='id')
# data.nodes
data.remove_nodes_from([0, 4, 6, 7, 18, 28, 46, 48, 51, 69, 76, 103, 104])
data = nx.relabel.convert_node_labels_to_integers(data, first_label=0, ordering='default')
W = nx.adjacency_matrix(data)
df = pd.read_csv('/content/Poll_Books_polbooks_resource-level.csv')
df = df[df.Label != 0]
df = df.reset_index(drop=True)
y_train = df.pop('Label')
y_train = y_train.to_numpy()

from scipy.sparse import csr_matrix

W = W.toarray()

acc = np.zeros (20)
for i in range (20):
  y_train_prime = y_train.copy()
  ra = random.sample(range(0, 92), 90)
  for j in ra:
    y_train_prime[j]=0
  pred , gamma_t = RobustGC (W, y_train_prime , 0.1)
  acc[i] = accuracy (pred , y_train)
print (np.mean(acc))

97.93478260869566


In [7]:
data = nx.read_gml('/content/polbooks.gml', label='id')
# data.nodes
data.remove_nodes_from([0, 4, 6, 7, 18, 28, 46, 48, 51, 69, 76, 103, 104])
data = nx.relabel.convert_node_labels_to_integers(data, first_label=0, ordering='default')
W = nx.adjacency_matrix(data)
df = pd.read_csv('/content/Poll_Books_polbooks_resource-level.csv')
df = df[df.Label != 0]
df = df.reset_index(drop=True)
y_train = df.pop('Label')
y_train = y_train.to_numpy()

from scipy.sparse import csr_matrix

W = W.toarray()

acc = np.zeros (20)
for i in range (20):
  y_train_prime = y_train.copy()
  ra = random.sample(range(0, 92), 90)
  for j in ra:
    y_train_prime[j]=0
  pred , gamma_t = PF_RobustGC (W, y_train_prime)
  acc[i] = accuracy (pred , y_train)
print (np.mean(acc))

97.82608695652173
